In [1]:
import pandas as pd
import numpy as np
import utils
import xgboost as xgb
from collections import defaultdict

/home/m.plekhanov/tools/anaconda3/envs/work/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from importlib import reload
utils = reload(utils)

In [3]:
!ls ./pkls

bigrams_cnt.pkl
bigrams_norm_cnt.pkl
bm25_bigrams_norm.pkl
bm25_bigrams.pkl
bm25_char_trigrams_norm.pkl
bm25_char_trigrams.pkl
bm25_unigram_norm.pkl
bm25_unigram.pkl
char_trigrams_cnt.pkl
char_trigrams_norm_cnt.pkl
df_bm25_bigram_scores_norm.pkl
df_bm25_bigram_scores.pkl
df_bm25_bigram_scores_smooth_norm.pkl
df_bm25_bigram_scores_smooth.pkl
df_bm25_bigrams_scores.pkl
df_bm25_trigram_scores.pkl
df_bm25_trigram_scores_smooth.pkl
df_bm25_unigram_scores_norm.pkl
df_bm25_unigram_scores.pkl
df_bm25_unigram_scores_smooth_norm.pkl
df_bm25_unigram_scores_smooth_norm.pkl.bak
df_bm25_unigram_scores_smooth.pkl
df_coverage_scores.pkl
df_train_bm25_bigram_scores_norm.pkl
df_train_bm25_bigram_scores.pkl
df_train_bm25_bigram_scores_smooth_norm.pkl
df_train_bm25_bigram_scores_smooth.pkl
df_train_bm25_trigram_scores.pkl
df_train_bm25_trigram_scores_smooth.pkl
df_train_bm25_unigram_scores.pkl
df_train_bm25_unigram_scores_smooth.pkl
df_train_coverage_scores.pkl
index_norm.pkl
index.pkl
index_test.pkl
unig

In [4]:
df_bm25_unigram_scores = utils.load_from_file("pkls/df_bm25_unigram_scores.pkl")
df_bm25_unigram_scores_smooth = utils.load_from_file("pkls/df_bm25_unigram_scores_smooth.pkl")
df_bm25_bigram_scores = utils.load_from_file("pkls/df_bm25_bigram_scores.pkl")
df_bm25_bigram_scores_smooth = utils.load_from_file("pkls/df_bm25_bigram_scores_smooth.pkl")
#df_bm25_trigram_scores = utils.load_from_file("pkls/df_bm25_trigram_scores.pkl")
#df_bm25_trigram_scores_smooth = utils.load_from_file("pkls/df_bm25_trigram_scores_smooth.pkl")
df_coverage_scores = utils.load_from_file("pkls/df_coverage_scores.pkl")

ImportError: No module named 'pandas.indexes'

In [ ]:
test_data = df_bm25_unigram_scores
for another_data in [ df_bm25_unigram_scores_smooth,
                      df_bm25_bigram_scores,
                      df_bm25_bigram_scores_smooth,
                      df_bm25_trigram_scores,
                      df_bm25_trigram_scores_smooth,
                      df_coverage_scores]:
    for column in another_data.columns:
        if column=='query_id' or column=='doc_id':
            continue
        test_data[column] = another_data[column]

In [ ]:
df_train_bm25_unigram_scores = utils.load_from_file("pkls/df_train_bm25_unigram_scores.pkl")
df_train_bm25_unigram_scores_smooth = utils.load_from_file("pkls/df_train_bm25_unigram_scores_smooth.pkl")
df_train_bm25_bigram_scores = utils.load_from_file("pkls/df_train_bm25_bigram_scores.pkl")
df_train_bm25_bigram_scores_smooth = utils.load_from_file("pkls/df_train_bm25_bigram_scores_smooth.pkl")
df_train_bm25_trigram_scores = utils.load_from_file("pkls/df_train_bm25_trigram_scores.pkl")
df_train_bm25_trigram_scores_smooth = utils.load_from_file("pkls/df_train_bm25_trigram_scores_smooth.pkl")
df_train_coverage_scores = utils.load_from_file("pkls/df_train_coverage_scores.pkl")

In [ ]:
train_data = df_train_bm25_unigram_scores
for another_data in [ df_train_bm25_unigram_scores_smooth,
                      df_train_bm25_bigram_scores,
                      df_train_bm25_bigram_scores_smooth,
                      df_train_bm25_trigram_scores,
                      df_train_bm25_trigram_scores_smooth,
                      df_train_coverage_scores]:
    for column in another_data.columns:
        if column=='query_id' or column=='doc_id':
            continue
        train_data[column] = another_data[column]

In [ ]:
test_data.columns

In [ ]:
queries = utils.load_queries('data/queries_tr.tsv')
test_queries = utils.load_test('data/test.tsv')
train_queries = utils.load_train('data/clicks.train.tsv')

In [ ]:
train_data['target'] = 0
test_data['target'] = 0

In [ ]:
clicked_pairs = set()
for query_id, docs, clicks in train_queries:
    clicks_set = set(clicks)
    for doc_id in docs:
        if doc_id in clicks:
            #train_data.target[(train_data.query_id==query_id) & (train_data.doc_id==doc_id)] = 1
            clicked_pairs.add((query_id, doc_id))

In [ ]:
train_data.target = train_data.apply(lambda x: (x['query_id'], x['doc_id']) in clicked_pairs, axis=1)

In [ ]:
feature_columns = ['score_title_bm25', 'score_body_bm25',
       'score_title_bm25_smooth', 'score_body_bm25_smooth',
       'score_title_bm25_bigram', 'score_body_bm25_bigram',
       'score_title_bm25_bigram_smooth', 'score_body_bm25_bigram_smooth',
       'score_title_bm25_trigram', 'score_body_bm25_trigram',
       'score_title_bm25_trigram_smooth', 'score_body_bm25_trigram_smooth',
       'title_count', 'body_count', 'whole_count', 'title_ratio', 'body_ratio',
       'whole_ratio']

In [ ]:
feature_columns = ['score_title_bm25', 'score_body_bm25', 'score_whole_bm25', 
       #'score_title_bm25_smooth', 'score_body_bm25_smooth',
       #'score_title_bm25_bigram', 'score_body_bm25_bigram',
       #'score_title_bm25_bigram_smooth', 'score_body_bm25_bigram_smooth',
       #'score_title_bm25_trigram', 'score_body_bm25_trigram',
       #'score_title_bm25_trigram_smooth', 'score_body_bm25_trigram_smooth',
       'title_count', 'body_count', 'whole_count', 'title_ratio', 'body_ratio',
       'whole_ratio']

In [ ]:
dtrain = xgb.DMatrix(train_data[feature_columns], label=train_data.target)
train_groups = [len(docs) for query_id,docs,clicks in train_queries]
dtrain.set_group(train_groups)

In [ ]:
dtest = xgb.DMatrix(test_data[feature_columns], label=test_data.target)
test_groups = [len(docs) for query_id,docs in test_queries]
dtest.set_group(test_groups)

In [ ]:
param = {'max_depth':5, 'eta':1, 'silent':1, 'objective':'rank:pairwise',
         'eval_metric': ['ndcg@5']}

In [ ]:
watchlist  = [(dtest,'eval'), (dtrain,'train')]

In [ ]:
num_round = 500

In [ ]:
model = xgb.train(param, dtrain, num_round, watchlist)

In [ ]:
predictions = model.predict(dtest)

In [ ]:
predictions[0:50]

In [ ]:
i = 0
query_predictions = {}
for query_id, docs in test_queries:
    query_predictions[query_id] = np.array(predictions[i:i+len(docs)])
    i+=len(docs)

In [ ]:
result = []
for query_id, docs in test_queries:
    top_docs = np.array(docs)[np.flip(np.argsort(query_predictions[query_id]),0)[0:5]]
    for doc_id in top_docs:
        result.append((query_id, doc_id))

In [ ]:
df = pd.DataFrame(result, columns=['QueryId','DocID'])

In [ ]:
df.to_csv('submit.csv', index=False)

In [ ]:
#### Use only bm25 unigrams

In [ ]:
test_data.head(50)

In [ ]:
new_data = test_data[['query_id', 'doc_id']].copy()
new_data.doc_id = new_data.doc_id.astype(int)

In [ ]:
new_data['score_final'] = test_data.score_title_bm25 + test_data.score_body_bm25 #+ test_data.score_whole_bm25

In [ ]:
query_scores = defaultdict(list)
for record in new_data.iterrows():
    query_scores[record[1]['query_id']].append((int(record[1]['doc_id']), record[1]['score_final']))

In [ ]:
result = []
for query_id, docs in test_queries:
    docs, query_predictions = zip(*query_scores[query_id])
    top_docs = np.array(docs)[np.argsort(query_predictions)][::-1][0:5]
    for doc_id in top_docs:
        result.append((query_id, doc_id))

In [ ]:
df = pd.DataFrame(result, columns=['QueryId','DocID'])

In [ ]:
df.head()

In [ ]:
df.to_csv('submit4.csv', index=False)